# Biblioteca

In [3]:
!pip install psycopg2-binary
!pip install sqlalchemy sqlalchemy-schemadisplay pydot
!pip install pydot
import psycopg2
import pandas as pd
from pathlib import Path


# Utilidades

In [1]:
def get_dataframe_from_csv(file_path, delimiter=','):
    try:
        df = pd.read_csv(file_path, dtype=str, delimiter=delimiter)
        print(f"Arquivo CSV '{file_path}' lido com sucesso.")
        return df
    except Exception as e:
        print(f"Erro ao ler o arquivo CSV: {e}")
        return pd.DataFrame()
    
def insert_dataframe_to_table(conn, cur, df, sql_query, columns, table_name):
    """
    Insere dados de um DataFrame em uma tabela.
    
    Args:
        conn: Conexão com o banco
        cur: Cursor do banco
        df: DataFrame com os dados
        sql_query: Query SQL de inserção
        columns: Lista com nomes das colunas do DataFrame
        table_name: Nome da tabela para mensagens
    """
    if conn is not None and cur is not None and not df.empty:
        try:
            data_tuples = [tuple(row[col] for col in columns) for _, row in df.iterrows()]
            cur.executemany(sql_query, data_tuples)
            conn.commit()
            print(f"Dados inseridos na tabela '{table_name}' com sucesso. ({len(data_tuples)} registros)")
            return True
        except psycopg2.Error as e:
            print(f"Erro ao inserir dados na tabela '{table_name}': {e}")
            conn.rollback()
            return False
    else:
        print(f"Não foi possível inserir dados na tabela '{table_name}': conexão inválida ou DataFrame vazio.")
        return False

# Conexão

In [4]:
db_config = {
    "host": "localhost",
    "port": "5432",
    "database": "tebd_db",
    "user": "tebd",
    "password": "tebd"
}

conn = None
cur = None

# --- Conectar ao Banco de Dados ---
try:
    conn = psycopg2.connect(**db_config)
    print("Conexão com o PostgreSQL bem-sucedida!")
    cur = conn.cursor()
except psycopg2.Error as e:
    print(f"Erro ao conectar ao PostgreSQL: {e}")


Conexão com o PostgreSQL bem-sucedida!


# Criação das tabelas

In [90]:
sql_create_municipio = """
    CREATE TABLE IF NOT EXISTS Municipio (
        municipio_id VARCHAR(6) PRIMARY KEY UNIQUE,
        municipio_nome VARCHAR(100)
    );
"""

sql_create_estabelecimento = """
    CREATE TABLE IF NOT EXISTS Estabelecimento (
        cnes_id VARCHAR(7) PRIMARY KEY UNIQUE,
        cnpj VARCHAR(15),
        municipio_id VARCHAR(6) REFERENCES Municipio(municipio_id)
    );
"""

sql_create_procedimento = """
    CREATE TABLE IF NOT EXISTS Procedimento (
        procedimento_id VARCHAR(10) PRIMARY KEY UNIQUE,
        procedimento_nome VARCHAR(255)
    );
"""

sql_create_cid = """
    CREATE TABLE IF NOT EXISTS Cid (
        cid_id VARCHAR(4) PRIMARY KEY UNIQUE,
        cid_nome VARCHAR(500)
    );
"""

sql_create_pessoa = """
    CREATE TABLE IF NOT EXISTS Pessoa (
        pessoa_id SERIAL PRIMARY KEY UNIQUE,
        nasc DATE,
        sexo VARCHAR(1),
        idade NUMERIC(2),
        raca_cor VARCHAR(4),
        etnia VARCHAR(4),
        cod_idade VARCHAR(1)
    );
"""

sql_create_internamento = """
    CREATE TABLE IF NOT EXISTS Internamento (
        internamento_id VARCHAR(13) PRIMARY KEY UNIQUE,
        qt_diarias NUMERIC(3),
        val_sh NUMERIC(13, 2),
        val_sp NUMERIC(13, 2),
        val_tot NUMERIC(14, 2),
        dt_inter DATE,
        morte NUMERIC(1),
        cid_morte VARCHAR(4),
        cnes_id VARCHAR(7) REFERENCES Estabelecimento(cnes_id),
        procedimento_id VARCHAR(10) REFERENCES Procedimento(procedimento_id),
        cid_id VARCHAR(4) REFERENCES Cid(cid_id),
        pessoa_id INT REFERENCES Pessoa(pessoa_id)
    );
"""

if conn is not None and cur is not None:
    try:
        cur.execute(sql_create_municipio)
        print("Tabela 'Municipio' criada com sucesso.")

        cur.execute(sql_create_estabelecimento)
        print("Tabela 'Estabelecimento' criada com sucesso.")

        cur.execute(sql_create_procedimento)
        print("Tabela 'Procedimento' criada com sucesso.")

        cur.execute(sql_create_cid)
        print("Tabela 'Cid' criada com sucesso.")

        cur.execute(sql_create_pessoa)
        print("Tabela 'Pessoa' criada com sucesso.")

        cur.execute(sql_create_internamento)
        print("Tabela 'Internamento' criada com sucesso.")

        conn.commit()
    except psycopg2.Error as e:
        print(f"Erro ao criar as tabelas: {e}")
        conn.rollback()

Tabela 'Municipio' criada com sucesso.
Tabela 'Estabelecimento' criada com sucesso.
Tabela 'Procedimento' criada com sucesso.
Tabela 'Cid' criada com sucesso.
Tabela 'Pessoa' criada com sucesso.
Tabela 'Internamento' criada com sucesso.


# Inserção nas tabelas

## Municipios

In [5]:
df_municipios = get_dataframe_from_csv('data/municipios_tratado.csv')

Arquivo CSV 'data/municipios_tratado.csv' lido com sucesso.


In [6]:
sql_insert_municipio = """
    INSERT INTO Municipio (municipio_id, municipio_nome)
    VALUES (%s, %s)
    ON CONFLICT (municipio_id) DO NOTHING;
"""

_ = insert_dataframe_to_table(
    conn, cur, df_municipios, 
    sql_insert_municipio, 
    ['codigo', 'municipio'], 
    'Municipio'
)

Dados inseridos na tabela 'Municipio' com sucesso. (185 registros)


## Estabelecimentos

In [7]:
df_estabelecimentos = get_dataframe_from_csv('data/CNES_tratado.csv')

Arquivo CSV 'data/CNES_tratado.csv' lido com sucesso.


In [8]:
sql_insert_estabelecimento = """
    INSERT INTO Estabelecimento (cnes_id, cnpj, municipio_id)
    VALUES (%s, %s, %s)
    ON CONFLICT (cnes_id) DO NOTHING;
"""

_ = insert_dataframe_to_table(
    conn, cur, df_estabelecimentos, 
    sql_insert_estabelecimento, 
    ['CNES', 'CPF_CNPJ', 'CODUFMUN'], 
    'Estabelecimento'
)

Dados inseridos na tabela 'Estabelecimento' com sucesso. (14586 registros)


## Procedimentos

In [9]:
df_procedimentos = get_dataframe_from_csv('data/procedimentos_tratado.csv', delimiter=';')

Arquivo CSV 'data/procedimentos_tratado.csv' lido com sucesso.


In [10]:
sql_insert_procedimento = """
    INSERT INTO Procedimento (procedimento_id, procedimento_nome)
    VALUES (%s, %s)
    ON CONFLICT (procedimento_id) DO NOTHING;
"""

_ = insert_dataframe_to_table(
    conn, cur, df_procedimentos, 
    sql_insert_procedimento, 
    ['codproc', 'nome'], 
    'Procedimento'
)


Dados inseridos na tabela 'Procedimento' com sucesso. (5279 registros)


## Cid

In [11]:
df_cid = get_dataframe_from_csv('data/cid_tratado.csv', delimiter=';')

Arquivo CSV 'data/cid_tratado.csv' lido com sucesso.


In [13]:
sql_insert_cid = """
    INSERT INTO Cid (cid_id, cid_nome)
    VALUES (%s, %s)
    ON CONFLICT (cid_id) DO NOTHING;
"""
_ = insert_dataframe_to_table(
    conn, cur, df_cid,
    sql_insert_cid,
    ['codigo', 'descricao'],
    'Cid'
)

Dados inseridos na tabela 'Cid' com sucesso. (14231 registros)


## Pessoa e Internamento

O dataframe para pessoa e internamento é o mesmo.

In [19]:
DEBUG = False

df_internamentos_list = []

if DEBUG:
    df_internamento = get_dataframe_from_csv('data/arquivo_500linhas.csv')
    df_internamentos_list.append(df_internamento)
else:
    df_internamento = get_dataframe_from_csv('data/sus/selected_data.csv')
    df_internamentos_list.append(df_internamento.head(500))


Arquivo CSV 'data/sus/selected_data.csv' lido com sucesso.


In [20]:
sql_insert_pessoa = """
    INSERT INTO Pessoa (nasc, sexo, idade, raca_cor, etnia, cod_idade)
    VALUES (%s, %s, %s, %s, %s, %s)
    RETURNING pessoa_id;
"""

sql_insert_internamento = """
    INSERT INTO Internamento (
        internamento_id, qt_diarias, val_sh, val_sp, val_tot, dt_inter, 
        morte, cid_morte, cnes_id, procedimento_id, cid_id, pessoa_id
    )
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    ON CONFLICT (internamento_id) DO NOTHING;
"""

if conn is None or cur is None:
    print("Conexão com o banco de dados não estabelecida. Não é possível inserir dados na tabela 'Internamento'.")
    exit(1)

for df_internamento in df_internamentos_list:
    if not df_internamento.empty:
        try:
            for _, row in df_internamento.iterrows():
                cur.execute(sql_insert_pessoa, (
                    row['NASC'], row['SEXO'], row['IDADE'], 
                    row['RACA_COR'], row['ETNIA'], row['COD_IDADE']
                ))
                pessoa_id = cur.fetchone()[0]

                # Garantir que PROC_REA tenha sempre 10 dígitos
                proc_rea = str(row['PROC_REA']).zfill(10)

                cur.execute(sql_insert_internamento, (
                    row['N_AIH'], row['QT_DIARIAS'], row['VAL_SH'], 
                    row['VAL_SP'], row['VAL_TOT'], row['DT_INTER'], 
                    row['MORTE'], row['CID_MORTE'], row['CNES'], 
                    proc_rea, row['DIAG_PRINC'], pessoa_id
                ))
            conn.commit()
            print(f"Dados inseridos na tabela 'Internamento' com sucesso. ({len(df_internamento)} registros)")
        except psycopg2.Error as e:
            print(f"Erro ao inserir dados na tabela 'Internamento': {e}")
            conn.rollback()
    else:
        print(f"DataFrame: {df_internamento} está vazio. Nenhum dado será inserido na tabela 'Internamento'.")


Erro ao inserir dados na tabela 'Internamento': insert or update on table "internamento" violates foreign key constraint "internamento_cid_id_fkey"
DETAIL:  Key (cid_id)=(N17) is not present in table "cid".



# Preview output

In [7]:
# Função para executar consultas e exibir resultados
def execute_query_and_display(query, title, limit=10):
    """
    Executa uma consulta SQL e exibe os resultados de forma formatada
    """
    if conn is not None and cur is not None:
        try:
            cur.execute(query)
            results = cur.fetchall()
            columns = [desc[0] for desc in cur.description]
            
            print(f"\n{'='*60}")
            print(f"{title}")
            print(f"{'='*60}")
            
            if results:
                # Criar DataFrame para melhor visualização
                df_result = pd.DataFrame(results, columns=columns)
                print(f"Total de registros: {len(df_result)}")
                print(f"\nPrimeiros {min(limit, len(df_result))} resultados:")
                print(df_result.head(limit).to_string(index=False))
                return df_result
            else:
                print("Nenhum resultado encontrado.")
                return pd.DataFrame()
        except psycopg2.Error as e:
            print(f"Erro ao executar consulta: {e}")
            return pd.DataFrame()
    else:
        print("Conexão com o banco não estabelecida.")
        return pd.DataFrame()

# Consultas Analíticas para Dashboard

Aqui vamos criar várias consultas que demonstram diferentes perspectivas dos dados de internação do DATASUS, úteis para visualizações em dashboard.

## 1. Análise Temporal de Internações

In [8]:
# Internações por mês/ano
query_internacoes_tempo = """
    SELECT 
        EXTRACT(YEAR FROM dt_inter) AS ano,
        EXTRACT(MONTH FROM dt_inter) AS mes,
        COUNT(*) AS total_internacoes,
        AVG(qt_diarias) AS media_diarias,
        SUM(val_tot) AS valor_total
    FROM Internamento 
    WHERE dt_inter IS NOT NULL
    GROUP BY EXTRACT(YEAR FROM dt_inter), EXTRACT(MONTH FROM dt_inter)
    ORDER BY ano, mes;
"""

df_tempo = execute_query_and_display(query_internacoes_tempo, "INTERNAÇÕES POR PERÍODO")


INTERNAÇÕES POR PERÍODO
Total de registros: 4

Primeiros 4 resultados:
 ano mes  total_internacoes      media_diarias valor_total
2018  10                  7 9.4285714285714286     3719.40
2018  11                 35 3.6000000000000000    25204.90
2018  12                164 7.2317073170731707   231865.87
2019   1                294 2.8333333333333333   190454.64


## 2. Top Diagnósticos (CID) mais Frequentes

In [9]:
# Top 15 diagnósticos mais frequentes
query_top_cid = """
    SELECT 
        c.cid_id,
        c.cid_nome,
        COUNT(i.internamento_id) AS total_casos,
        AVG(i.val_tot) AS valor_medio,
        AVG(i.qt_diarias) AS media_diarias,
        COUNT(CASE WHEN i.morte = 1 THEN 1 END) AS obitos,
        ROUND(
            (COUNT(CASE WHEN i.morte = 1 THEN 1 END) * 100.0 / COUNT(i.internamento_id)), 2
        ) AS taxa_mortalidade_pct
    FROM Cid c
    JOIN Internamento i ON c.cid_id = i.cid_id
    GROUP BY c.cid_id, c.cid_nome
    ORDER BY total_casos DESC
    LIMIT 15;
"""

df_top_cid = execute_query_and_display(query_top_cid, "TOP 15 DIAGNÓSTICOS MAIS FREQUENTES")


TOP 15 DIAGNÓSTICOS MAIS FREQUENTES
Total de registros: 15

Primeiros 10 resultados:
cid_id                                                   cid_nome  total_casos          valor_medio          media_diarias  obitos taxa_mortalidade_pct
  O800                                  Parto espontâneo cefálico           32 527.3300000000000000     1.7812500000000000       0                 0.00
  O801                                   Parto espontâneo pélvico           16 472.4437500000000000     1.8750000000000000       0                 0.00
  O034           Aborto espontâneo - incompleto, sem complicações           15 195.6900000000000000     2.0666666666666667       0                 0.00
  S525                     Fratura da extremidade distal do rádio           14 606.9457142857142857     4.2857142857142857       0                 0.00
  H251                                     Catarata senil nuclear           14 531.6000000000000000                  0E-20       0                 0.00
  

## 3. Análise por Faixa Etária e Sexo

In [10]:
# Distribuição por faixa etária e sexo
query_idade_sexo = """
    SELECT 
        CASE 
            WHEN p.idade BETWEEN 0 AND 17 THEN '0-17 anos'
            WHEN p.idade BETWEEN 18 AND 39 THEN '18-39 anos'
            WHEN p.idade BETWEEN 40 AND 59 THEN '40-59 anos'
            WHEN p.idade BETWEEN 60 AND 79 THEN '60-79 anos'
            WHEN p.idade >= 80 THEN '80+ anos'
            ELSE 'Não informado'
        END AS faixa_etaria,
        CASE 
            WHEN p.sexo = 'M' THEN 'Masculino'
            WHEN p.sexo = 'F' THEN 'Feminino'
            ELSE 'Não informado'
        END AS sexo,
        COUNT(i.internamento_id) AS total_internacoes,
        AVG(i.val_tot) AS valor_medio,
        AVG(i.qt_diarias) AS media_diarias
    FROM Pessoa p
    JOIN Internamento i ON p.pessoa_id = i.pessoa_id
    WHERE p.idade IS NOT NULL AND p.sexo IS NOT NULL
    GROUP BY faixa_etaria, sexo
    ORDER BY faixa_etaria, sexo;
"""

df_idade_sexo = execute_query_and_display(query_idade_sexo, "DISTRIBUIÇÃO POR FAIXA ETÁRIA E SEXO")


DISTRIBUIÇÃO POR FAIXA ETÁRIA E SEXO
Total de registros: 10

Primeiros 10 resultados:
faixa_etaria          sexo  total_internacoes           valor_medio      media_diarias
   0-17 anos Não informado                 44  365.2747727272727273 3.0000000000000000
   0-17 anos Não informado                 36  416.8341666666666667 3.4722222222222222
  18-39 anos Não informado                116  624.7629310344827586 2.6982758620689655
  18-39 anos Não informado                 56 1085.8317857142857143 4.2678571428571429
  40-59 anos Não informado                 43 1169.8190697674418605 6.6511627906976744
  40-59 anos Não informado                 76 1453.7218421052631579 5.1447368421052632
  60-79 anos Não informado                 63 1093.5252380952380952 5.2539682539682540
  60-79 anos Não informado                 32  898.3878125000000000 5.7187500000000000
    80+ anos Não informado                 15  574.4800000000000000 5.0666666666666667
    80+ anos Não informado                 

## 4. Ranking de Municípios por Volume de Internações

In [12]:
# Top 10 municípios com mais internações
query_ranking_municipios = """
    SELECT 
        m.municipio_nome,
        COUNT(i.internamento_id) AS total_internacoes,
        SUM(i.val_tot) AS valor_total,
        AVG(i.val_tot) AS valor_medio_internacao,
        COUNT(DISTINCT e.cnes_id) AS num_estabelecimentos
    FROM Municipio m
    JOIN Estabelecimento e ON m.municipio_id = e.municipio_id
    JOIN Internamento i ON e.cnes_id = i.cnes_id
    GROUP BY m.municipio_id, m.municipio_nome
    ORDER BY total_internacoes DESC
    LIMIT 10;
"""

df_ranking_municipios = execute_query_and_display(query_ranking_municipios, "TOP 10 MUNICÍPIOS POR VOLUME DE INTERNAÇÕES")


TOP 10 MUNICÍPIOS POR VOLUME DE INTERNAÇÕES
Total de registros: 8

Primeiros 8 resultados:
municipio_nome  total_internacoes valor_total valor_medio_internacao  num_estabelecimentos
        Recife                122   253025.54  2073.9798360655737705                     1
     Arcoverde                 99    51880.44   524.0448484848484848                     1
     Garanhuns                 83    35839.78   431.8045783132530120                     1
   Petrolândia                 60    25929.62   432.1603333333333333                     1
     Petrolina                 51    22229.65   435.8754901960784314                     2
        Moreno                 40    48310.76  1207.7690000000000000                     1
        Buíque                 35    10116.41   289.0402857142857143                     1
       Caruaru                 10     3912.61   391.2610000000000000                     1


## 5. Análise de Procedimentos Mais Custosos

In [13]:
# Top 10 procedimentos mais custosos (por valor total)
query_procedimentos_custosos = """
    SELECT 
        p.procedimento_nome,
        COUNT(i.internamento_id) AS total_realizacoes,
        SUM(i.val_tot) AS valor_total_gasto,
        AVG(i.val_tot) AS valor_medio_procedimento,
        AVG(i.qt_diarias) AS media_dias_internacao
    FROM Procedimento p
    JOIN Internamento i ON p.procedimento_id = i.procedimento_id
    GROUP BY p.procedimento_id, p.procedimento_nome
    HAVING COUNT(i.internamento_id) >= 2  -- Filtra procedimentos com pelo menos 2 casos
    ORDER BY valor_total_gasto DESC
    LIMIT 10;
"""

df_procedimentos_custosos = execute_query_and_display(query_procedimentos_custosos, "TOP 10 PROCEDIMENTOS MAIS CUSTOSOS")


TOP 10 PROCEDIMENTOS MAIS CUSTOSOS
Total de registros: 10

Primeiros 10 resultados:
                                                                                                                                                                                                                                         procedimento_nome  total_realizacoes valor_total_gasto valor_medio_procedimento  media_dias_internacao
TRATAMENTO C/ CIRURGIAS MULTIPLAS                                                                                                                                                                                                                                          29          86787.97    2992.6886206896551724     6.3793103448275862
PARTO NORMAL                                                                                                                                                                                                                                       

## 6. Análise de Mortalidade por Diagnóstico

In [14]:
# Diagnósticos com maiores taxas de mortalidade (mínimo 10 casos)
query_mortalidade_cid = """
    SELECT 
        c.cid_id,
        c.cid_nome,
        COUNT(i.internamento_id) AS total_casos,
        COUNT(CASE WHEN i.morte = 1 THEN 1 END) AS total_obitos,
        ROUND(
            (COUNT(CASE WHEN i.morte = 1 THEN 1 END) * 100.0 / COUNT(i.internamento_id)), 2
        ) AS taxa_mortalidade_pct,
        AVG(i.qt_diarias) AS media_dias_internacao
    FROM Cid c
    JOIN Internamento i ON c.cid_id = i.cid_id
    GROUP BY c.cid_id, c.cid_nome
    HAVING COUNT(i.internamento_id) >= 10  -- Mínimo de 10 casos para análise estatística
    ORDER BY taxa_mortalidade_pct DESC
    LIMIT 10;
"""

df_mortalidade_cid = execute_query_and_display(query_mortalidade_cid, "TOP 10 DIAGNÓSTICOS COM MAIOR TAXA DE MORTALIDADE")


TOP 10 DIAGNÓSTICOS COM MAIOR TAXA DE MORTALIDADE
Total de registros: 10

Primeiros 10 resultados:
cid_id                                                   cid_nome  total_casos  total_obitos taxa_mortalidade_pct  media_dias_internacao
  O034           Aborto espontâneo - incompleto, sem complicações           15             0                 0.00     2.0666666666666667
  O821                          Parto por cesariana de emergência           10             0                 0.00     2.8000000000000000
  L989 Afecções da pele e do tecido subcutâneo, não especificados           10             0                 0.00 0.30000000000000000000
  O801                                   Parto espontâneo pélvico           16             0                 0.00     1.8750000000000000
  T141              Ferimento de região não especificada do corpo           10             0                 0.00     7.3000000000000000
  H251                                     Catarata senil nuclear           14

## 7. Análise por Estabelecimentos (CNES)

In [20]:
# Top 10 estabelecimentos por volume de internações
query_ranking_estabelecimentos = """
    SELECT 
        e.cnes_id,
        m.municipio_nome,
        COUNT(i.internamento_id) AS total_internacoes,
        SUM(i.val_tot) AS valor_total_movimentado,
        AVG(i.val_tot) AS valor_medio_internacao,
        AVG(i.qt_diarias) AS media_dias_internacao,
        COUNT(CASE WHEN i.morte = 1 THEN 1 END) AS total_obitos,
        ROUND(
            (COUNT(CASE WHEN i.morte = 1 THEN 1 END) * 100.0 / COUNT(i.internamento_id)), 2
        ) AS taxa_mortalidade_pct
    FROM Estabelecimento e
    JOIN Municipio m ON e.municipio_id = m.municipio_id
    JOIN Internamento i ON e.cnes_id = i.cnes_id
    GROUP BY e.cnes_id, m.municipio_nome
    ORDER BY total_internacoes DESC
    LIMIT 10;
"""

df_ranking_estabelecimentos = execute_query_and_display(query_ranking_estabelecimentos, "TOP 10 ESTABELECIMENTOS POR VOLUME DE INTERNAÇÕES")

Erro ao executar consulta: current transaction is aborted, commands ignored until end of transaction block



## 8. Análise de Tempo de Internação

In [17]:
# Análise de distribuição do tempo de internação
query_tempo_internacao = """
    SELECT 
        CASE 
            WHEN qt_diarias = 0 THEN 'Menos de 1 dia'
            WHEN qt_diarias BETWEEN 1 AND 3 THEN '1-3 dias'
            WHEN qt_diarias BETWEEN 4 AND 7 THEN '4-7 dias'
            WHEN qt_diarias BETWEEN 8 AND 15 THEN '8-15 dias'
            WHEN qt_diarias BETWEEN 16 AND 30 THEN '16-30 dias'
            WHEN qt_diarias > 30 THEN 'Mais de 30 dias'
        END AS faixa_dias,
        COUNT(*) AS total_internacoes,
        AVG(val_tot) AS valor_medio,
        COUNT(CASE WHEN morte = 1 THEN 1 END) AS total_obitos,
        ROUND(
            (COUNT(CASE WHEN morte = 1 THEN 1 END) * 100.0 / COUNT(*)), 2
        ) AS taxa_mortalidade_pct
    FROM Internamento
    WHERE qt_diarias IS NOT NULL
    GROUP BY faixa_dias
    ORDER BY 
        CASE 
            WHEN qt_diarias = 0 THEN 1
            WHEN qt_diarias BETWEEN 1 AND 3 THEN 2
            WHEN qt_diarias BETWEEN 4 AND 7 THEN 3
            WHEN qt_diarias BETWEEN 8 AND 15 THEN 4
            WHEN qt_diarias BETWEEN 16 AND 30 THEN 5
            WHEN qt_diarias > 30 THEN 6
        END;
"""

df_tempo_internacao = execute_query_and_display(query_tempo_internacao, "DISTRIBUIÇÃO DO TEMPO DE INTERNAÇÃO")

Erro ao executar consulta: current transaction is aborted, commands ignored until end of transaction block



## 9. Análise Resumo - KPIs Gerais

In [18]:
# KPIs gerais do sistema
query_kpis_gerais = """
    SELECT 
        'Indicadores Gerais' AS categoria,
        COUNT(DISTINCT i.internamento_id) AS total_internacoes,
        COUNT(DISTINCT e.cnes_id) AS total_estabelecimentos,
        COUNT(DISTINCT m.municipio_id) AS total_municipios,
        COUNT(DISTINCT c.cid_id) AS total_diagnosticos_utilizados,
        COUNT(DISTINCT p.procedimento_id) AS total_procedimentos_utilizados,
        ROUND(SUM(i.val_tot), 2) AS valor_total_gasto,
        ROUND(AVG(i.val_tot), 2) AS valor_medio_internacao,
        ROUND(AVG(i.qt_diarias), 1) AS media_dias_internacao,
        COUNT(CASE WHEN i.morte = 1 THEN 1 END) AS total_obitos,
        ROUND(
            (COUNT(CASE WHEN i.morte = 1 THEN 1 END) * 100.0 / COUNT(i.internamento_id)), 2
        ) AS taxa_mortalidade_geral_pct
    FROM Internamento i
    JOIN Estabelecimento e ON i.cnes_id = e.cnes_id
    JOIN Municipio m ON e.municipio_id = m.municipio_id
    JOIN Cid c ON i.cid_id = c.cid_id
    JOIN Procedimento p ON i.procedimento_id = p.procedimento_id;
"""

df_kpis_gerais = execute_query_and_display(query_kpis_gerais, "KPIs GERAIS DO SISTEMA", limit=20)

Erro ao executar consulta: current transaction is aborted, commands ignored until end of transaction block



## 10. Consulta Customizada - Análise Combinada

In [19]:
# Análise combinada: diagnóstico + procedimento + faixa etária
query_analise_combinada = """
    SELECT 
        c.cid_nome,
        proc.procedimento_nome,
        CASE 
            WHEN pe.idade BETWEEN 0 AND 17 THEN '0-17 anos'
            WHEN pe.idade BETWEEN 18 AND 39 THEN '18-39 anos'
            WHEN pe.idade BETWEEN 40 AND 59 THEN '40-59 anos'
            WHEN pe.idade BETWEEN 60 AND 79 THEN '60-79 anos'
            WHEN pe.idade >= 80 THEN '80+ anos'
            ELSE 'Não informado'
        END AS faixa_etaria,
        COUNT(i.internamento_id) AS total_casos,
        AVG(i.val_tot) AS valor_medio,
        AVG(i.qt_diarias) AS media_dias,
        COUNT(CASE WHEN i.morte = 1 THEN 1 END) AS obitos,
        ROUND(
            (COUNT(CASE WHEN i.morte = 1 THEN 1 END) * 100.0 / COUNT(i.internamento_id)), 2
        ) AS taxa_mortalidade_pct
    FROM Internamento i
    JOIN Cid c ON i.cid_id = c.cid_id
    JOIN Procedimento proc ON i.procedimento_id = proc.procedimento_id
    JOIN Pessoa pe ON i.pessoa_id = pe.pessoa_id
    WHERE pe.idade IS NOT NULL
    GROUP BY c.cid_nome, proc.procedimento_nome, faixa_etaria
    HAVING COUNT(i.internamento_id) >= 3  -- Pelo menos 3 casos para análise
    ORDER BY total_casos DESC
    LIMIT 15;
"""

df_analise_combinada = execute_query_and_display(query_analise_combinada, "ANÁLISE COMBINADA: DIAGNÓSTICO + PROCEDIMENTO + FAIXA ETÁRIA")

Erro ao executar consulta: current transaction is aborted, commands ignored until end of transaction block



## Resumo das Análises Criadas

As consultas acima cobrem várias perspectivas importantes dos dados de internação:

### Para Dashboard - Visualizações Sugeridas:
1. **Série Temporal**: Gráfico de linha mostrando evolução das internações ao longo do tempo
2. **Ranking CID**: Gráfico de barras com os diagnósticos mais frequentes
3. **Demografia**: Pirâmide etária ou gráfico de barras por faixa etária/sexo
4. **Mapa Geográfico**: Heatmap dos municípios por volume de internações
5. **Análise Financeira**: Gráficos de custo por procedimento
6. **Indicadores de Qualidade**: Dashboards de mortalidade e tempo médio de internação
7. **Performance Hospitalar**: Ranking de estabelecimentos
8. **Análise Integrada**: Tabelas e gráficos combinando múltiplas dimensões

### KPIs Principais:
- Total de internações
- Valor total movimentado
- Taxa de mortalidade geral
- Tempo médio de internação
- Número de estabelecimentos/municípios atendidos